## get word dict

In [1]:
import os
import glob

N = 25
def count_words():
    root_dir = os.getcwd()
    ham_dict = []    # dictionary of ham mails
    ham_cont = []    # word frequency in a mail
    ham_appe = []    # word appearance times across mails
    spam_dict = []
    spam_cont = []
    spam_appe = []
    os.chdir('./train_data/ham')
    data_train = glob.glob('*')
    for name in data_train:
        with open(name) as text_file:
            text = text_file.read()
            temp_dict = []
            temp_cont = []
            for word in text.split():
                if word.isdigit():
                    word = str(len(word))
                elif word.isalpha():
                    word = word.lower()
                else:
                    #word = "*"
                    continue
                if word in temp_dict:
                    temp_cont[temp_dict.index(word)] += 1
                else:
                    temp_dict.append(word)
                    temp_cont.append(1)
            for word in temp_dict:
                temp_cont[temp_dict.index(word)] /= (sum(temp_cont) + 1)
                if word in ham_dict:
                    ham_cont[ham_dict.index(word)] += temp_cont[temp_dict.index(word)]
                    ham_appe[ham_dict.index(word)] += 1
                else:
                    ham_dict.append(word)
                    ham_cont.append(temp_cont[temp_dict.index(word)])
                    ham_appe.append(1)
    os.chdir(root_dir)
    
    os.chdir('./train_data/spam')
    data_train = glob.glob('*')
    for name in data_train:
        with open(name) as text_file:
            text = text_file.read()
            temp_dict = []
            temp_cont = []
            for word in text.split():
                if word.isdigit():
                    word = str(len(word))
                elif word.isalpha():
                    word = word.lower()
                else:
                    #word = "*"
                    continue
                if word in temp_dict:
                    temp_cont[temp_dict.index(word)] += 1
                else:
                    temp_dict.append(word)
                    temp_cont.append(1)
            for word in temp_dict:
                temp_cont[temp_dict.index(word)] /= (sum(temp_cont) + 1)
                if word in spam_dict:
                    spam_cont[spam_dict.index(word)] += temp_cont[temp_dict.index(word)]
                    spam_appe[spam_dict.index(word)] += 1
                else:
                    spam_dict.append(word)
                    spam_cont.append(temp_cont[temp_dict.index(word)])
                    spam_appe.append(1)
    os.chdir(root_dir)
    
    return ham_dict, ham_cont, spam_dict, spam_cont, ham_appe, spam_appe
    
    
def merge_dict(dict1, dict2):
    dict_out = dict1
    for word in dict2:
        if word not in dict_out:
            dict_out.append(word)
    
    return dict_out

In [2]:
ham_dict, ham_cont, spam_dict, spam_cont, ham_appe, spam_appe = count_words()
ham_len = len(ham_dict)
spam_len = len(spam_dict)
print(ham_len)
print(spam_len)

15150
33510


In [3]:
words = merge_dict(ham_dict, spam_dict)

from heapq import nlargest

spam_top = nlargest(N, spam_cont)
spam_ = []
for i in range(N):
    spam_.append(spam_dict[spam_cont.index(spam_top[i])])
    
ham_top = nlargest(N, ham_cont)
ham_ = []
for i in range(N):
    ham_.append(ham_dict[ham_cont.index(ham_top[i])])
    
words = merge_dict(ham_, spam_)
print(words)

import numpy as np
from heapq import nlargest

word_dict = []
word_diff = []
word_diff_abs = []

for i in range(len(ham_dict)):
    ham_cont[i] *= np.log(ham_len/ham_appe[i])/ham_len
    # ham_cont[i] /= float(ham_len)
    word_dict.append(ham_dict[i])
    if ham_dict[i] in spam_dict:
        index = spam_dict.index(ham_dict[i])
        spam_cont[index] *= np.log(spam_len/spam_appe[index])/spam_len
        # spam_cont[index] /= float(spam_len)
        word_diff.append(ham_cont[i]-spam_cont[index])
        word_diff_abs.append(np.abs(ham_cont[i]-spam_cont[index]))
        del spam_cont[index]
        del spam_dict[index]
        del spam_appe[index]
    else:
        word_diff.append(ham_cont[i])
        word_diff_abs.append(ham_cont[i])
        
for i in range(len(spam_dict)):
    spam_cont[i] *= np.log(spam_len/spam_appe[i])/spam_len
    # spam_cont[i] /= float(spam_len)
    word_dict.append(spam_dict[i])
    word_diff.append(-spam_cont[i])
    word_diff_abs.append(spam_cont[i])
    


top_10_diff = nlargest(N, word_diff_abs)
words = []
for i in range(N):
    words.append(word_dict[word_diff_abs.index(top_10_diff[i])])
    print(word_diff[word_dict.index(words[i])], "   ", words[i])
    

## load data

In [4]:
import numpy as np
import os
import glob

def load_train(seed=100):
    root_dir = os.getcwd()
    train_data = []
    train_label = []
    test_data = []
    test_label = []
    os.chdir('./train_data/ham')
    data_train = glob.glob('*')
    for name in data_train:
        file = []
        with open(name) as text_file:
            text = text_file.read()
            for word in text.split():
                if word.isdigit():
                    word = str(len(word))
                elif word.isalpha():
                    word = word.lower()
                else:
                    word = "*"
                file.append(word)
        train_data.append(file)
        train_label.append(0)
    os.chdir(root_dir)
    
    os.chdir('./train_data/spam')
    data_train = glob.glob('*')
    for name in data_train:
        with open(name) as text_file:
            file = []
            text = text_file.read()
            for word in text.split():
                if word.isdigit():
                    word = str(len(word))
                elif word.isalpha():
                    word = word.lower()
                else:
                    word = "*"
                file.append(word)
        train_data.append(file)
        train_label.append(1)
    os.chdir(root_dir)
    
    np.random.seed(100)
    np.random.shuffle(train_data)
    np.random.seed(100)
    np.random.shuffle(train_label)
    
    train_data = np.asarray(train_data)
    train_label = np.asarray(train_label)
    
    indice = np.random.choice(np.arange(train_data.shape[0]), int(train_data.shape[0]*0.1), replace = False)
    
    test_data = train_data[indice]
    test_label = train_label[indice]
    train_data = np.delete(train_data, indice)
    train_label = np.delete(train_label, indice)
    
    return train_data, train_label, test_data, test_label

def load_test():
    root_dir = os.getcwd()
    test_data = []
    test_label = []
    os.chdir('./test_data')
    for i in range(800):
        name = 'test_email_'+str(i+1)+'.txt'
        file = []
        with open(name) as text_file:
            text = text_file.read()
            for word in text.split():
                if word.isdigit():
                    word = str(len(word))
                elif word.isalpha():
                    word = word.lower()
                else:
                    word = "*"
                file.append(word)
        test_data.append(file)
        test_label.append(0)
    os.chdir(root_dir)
    
    return test_data, test_label

In [5]:
train_data, train_label, test_data, test_label = load_train()
final_data, final_label = load_test()

## train model and test

In [6]:
import numpy as np
import tensorflow as tf
import time

def my_training(X_train, y_train, X_val, y_val, final_data, final_label, seed=100, learning_rate=1e-3, epoch=20, batch_size=500, verbose=False, pre_trained_model=None):
    print("learning_rate={}".format(learning_rate))

    with tf.name_scope('inputs'):
        xs = tf.placeholder(shape=[None, len(words)], dtype=tf.float32)
        ys = tf.placeholder(shape=[None, ], dtype=tf.int64)

    output, loss = build_model(xs, ys)
    
    iters = int(len(X_train) / batch_size)
    step = train_step(loss, learning_rate)
    print('number of batches for training: {}'.format(iters))

    eve, pre = evaluate(output, ys)
    
    iter_total = 0
    best_acc = 0
    cur_model_name = 'myModel_{}'.format(int(time.time()))
    prediction = 0
    vali = 0

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epc in range(epoch):
            print("epoch {} ".format(epc + 1))

            for itr in range(iters):
                iter_total += 1

                training_batch_x = X_train[itr * batch_size: (1 + itr) * batch_size]
                training_batch_y = y_train[itr * batch_size: (1 + itr) * batch_size]

                _, cur_loss = sess.run([step, loss], feed_dict={xs: training_batch_x, ys: training_batch_y})

                if iter_total % 100 == 0:
                    # do validation
                    valid_eve = sess.run(eve, feed_dict={xs: X_val, ys: y_val})
                    valid_acc = 100 - valid_eve * 100 / y_val.shape[0]
                    if verbose:
                        print('{}/{} loss: {} validation accuracy : {}%'.format(
                            batch_size * (itr + 1),
                            X_train.shape[0],
                            cur_loss,
                            valid_acc))
                    
                    # when achieve the best validation accuracy, we store the model paramters
                    print('current accuracy! iteration:{} accuracy: {}%'.format(iter_total, valid_acc))
                    if valid_acc > best_acc:
                        print('Best validation accuracy! iteration:{} accuracy: {}%'.format(iter_total, valid_acc))
                        best_acc = valid_acc
                        prediction = sess.run(pre, feed_dict={xs: final_data, ys: final_label})
        
        
    print("Traning ends. The best valid accuracy is {}.".format(best_acc))
    
    return prediction


class fc_layer(object):
    def __init__(self, input_x, in_size, out_size, rand_seed=100, activation_function=None, index=0):
        with tf.variable_scope('fc_layer_%d' % index):
            with tf.name_scope('fc_kernel'):
                w_shape = [in_size, out_size]
                weight = tf.get_variable(name='fc_kernel_%d' % index, shape=w_shape,
                                         initializer=tf.glorot_uniform_initializer(seed=rand_seed))
                self.weight = weight

            with tf.variable_scope('fc_kernel'):
                b_shape = [out_size]
                bias = tf.get_variable(name='fc_bias_%d' % index, shape=b_shape,
                                       initializer=tf.glorot_uniform_initializer(seed=rand_seed))
                self.bias = bias

            cell_out = tf.add(tf.matmul(input_x, weight), bias)
            if activation_function is not None:
                cell_out = activation_function(cell_out)
            self.cell_out = cell_out
            
    def output(self):
        return self.cell_out
    
    
def evaluate(output, input_y):
    with tf.name_scope('evaluate'):
        pred = tf.argmax(output, axis=1)
        error_num = tf.count_nonzero(pred - input_y, name='error_num')
    return error_num, pred


def cross_entropy(output, input_y):
    with tf.name_scope('cross_entropy'):
        label = tf.one_hot(input_y, 5)
        ce = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=label, logits=output))

    return ce


def train_step(loss, learning_rate=1e-3):
    with tf.name_scope('train_step'):
        step = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.9, beta2=0.999, 
                                       epsilon=1e-08, use_locking=False, 
                                       name='Adam').minimize(loss)
        
    return step


def build_model(x_tf, y_tf):
    
    fc_layer1 = fc_layer(input_x=x_tf, in_size=len(words), out_size=150, activation_function=tf.nn.relu, index=1)
    drop_layer1 = tf.nn.dropout(fc_layer1.output(), 0.5)
    
    fc_layer2 = fc_layer(input_x=drop_layer1, in_size=150, out_size=50, activation_function=tf.nn.relu, index=2)
    drop_layer2 = tf.nn.dropout(fc_layer2.output(), 0.5)
    
    fc_layer3 = fc_layer(input_x=drop_layer2, in_size=50, out_size=2, activation_function=None, index=3)

    logits = fc_layer3.output()

    fc_w = [fc_layer1.weight, fc_layer2.weight, fc_layer3.weight]
    
    with tf.name_scope("loss"):
        l2_loss = tf.reduce_sum([tf.norm(w) for w in fc_w])
        label = tf.one_hot(y_tf, 2)
        cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=label, logits=logits), name='cross_entropy')
        loss = tf.add(cross_entropy_loss, 0.01 * l2_loss, name='loss')
    
    return logits, loss


def get_frequency(in_data, word_dict):
    out_frequency = []
    for sample in in_data:
        temp = np.zeros(len(words))
        count = 1
        for word in sample:
            count += 1
            if word in word_dict:
                temp[word_dict.index(word)] += 1
        for i in range(len(words)):
            temp[i] = float(temp[i]) / count
        out_frequency.append(temp)
        
    return out_frequency
                

In [7]:
X_train = get_frequency(train_data, words)
X_val = get_frequency(test_data, words)
X_test = get_frequency(final_data, words)

y_train = train_label
y_val = test_label
y_test = final_label

In [8]:
prediction = my_training(X_train, y_train, X_val, y_val, X_test, y_test, seed=100, learning_rate=1e-3, epoch=100, batch_size=100, verbose=False, pre_trained_model=None)

learning_rate=0.001
number of batches for training: 39
epoch 1 
epoch 2 
epoch 3 
current accuracy! iteration:100 accuracy: 68.8787185354691%
Best validation accuracy! iteration:100 accuracy: 68.8787185354691%
epoch 4 
epoch 5 
epoch 6 
current accuracy! iteration:200 accuracy: 77.57437070938215%
Best validation accuracy! iteration:200 accuracy: 77.57437070938215%
epoch 7 
epoch 8 
current accuracy! iteration:300 accuracy: 92.44851258581235%
Best validation accuracy! iteration:300 accuracy: 92.44851258581235%
epoch 9 
epoch 10 
epoch 11 
current accuracy! iteration:400 accuracy: 92.44851258581235%
epoch 12 
epoch 13 
current accuracy! iteration:500 accuracy: 94.05034324942791%
Best validation accuracy! iteration:500 accuracy: 94.05034324942791%
epoch 14 
epoch 15 
epoch 16 
current accuracy! iteration:600 accuracy: 94.27917620137299%
Best validation accuracy! iteration:600 accuracy: 94.27917620137299%
epoch 17 
epoch 18 
current accuracy! iteration:700 accuracy: 94.50800915331808%
Best

## write CSV

In [9]:
import csv
import os

with open('predicted.csv','w') as csvfile:
    fieldnames = ['email_id','labels']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()    
    for index,l in enumerate(prediction):
        filename = str(index+1)
        label = str(l)
        writer.writerow({'email_id': filename, 'labels': label})